# LC gauge transformation of Glasma using gauge fields
Done following Meijian's Mathematica notebook.

### Extract the gauge fields
First, let's extract the Glasma gauge fields $A^x(x,y,\tau)$ and $A^y(x,y,\tau)$ as logarithms of the corresponding gauge links. We will use the same simulation parameters as in the `Light cone gauge transformation of the Glasma fields.ipynb` notebook.

#### Simulation parameters

In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 2      
N = 128   
tau_sim = 1     
DTS = 8     

# Glasma fields
su_group = 'su3'
Qs = 2        
ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu         
uv = 10.0       

# TODO: Run more events
nevents = 1

#### Environment variables

In [2]:
import os

# I need to add this line to ask resources from a specific GPU, which is free. Our GPU server has no queing system
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
# Number of colors
Nc = su.NC
# Dimension of algebra 
Dg = su.GROUP_ELEMENTS

Using Numba
Using SU(3)
Using double precision


#### Simulation routine for evolving the Glasma fields
The gauge field is extracted as the logarithm of the gauge link $\boxed{A_i(x,y,\tau)=\,\mathrm{ln} \,U_i(x,y,\tau)}$ with $i=x,y$ and $g$ and $a$ absorbed in the numerical definition of the gauge field. Notice that the gauge field is not evaluated in the midpoint $x+a/2$ since this is already a $\mathcal{O}(a^2)$ correction. As extracted in the notebook [Extract the logatirhm of a SU(3) gauge link
](https://github.com/avramescudana/curraun/blob/wong/notebooks/Logarithm%20of%20a%20SU(3)%20gauge%20link.ipynb), the fastest and simplest way to extract the logarithm of a gauge link is using the anti-hermitian part $\boxed{\mathrm{ln}\,U=\dfrac{U-U^\dagger}{2}}$. It is equivalent to performing a Taylor series of the logarithm,

In [16]:
def ln_ah(ui):
    buf0 = su.dagger(ui)
    buf1 = su.mul_s(buf0, -1)
    buf2 = su.add(ui, buf1)
    # buf3 = su.mul_s(buf2, -0.5j)
    buf3 = su.mul_s(buf2, 0.5)
    ai = np.array(buf3)
    return ai

Alternatively, extract the logarithm of a gauge link as a Taylor series.

In [17]:
def ln_taylor(ui):
    buf1 = su.mlog(ui)
    # buf2 = su.mul_s(buf1, -1.0j)
    buf2 = su.mul_s(buf1, 1.0)
    ai = np.array(buf2)
    return ai

In [5]:
import pickle
from tqdm import tqdm

# Simulation routine
def simulate(): 

    # Derived parameters
    a = L / N
    E0 = N / L * hbarc
    DT = 1.0 / DTS
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    # Transverse gauge links
    ax_ah = su.GROUP_TYPE(np.zeros((maxt//DTS, N*N, Dg)))
    ax_taylor = su.GROUP_TYPE(np.zeros((maxt//DTS, N*N, Dg)))
    # ay = su.GROUP_TYPE(np.zeros((maxt//DTS, N*N, Dg)))

    with tqdm(total=maxt) as pbar:
        for t in range(maxt):            
            # Evolve Glasma fields
            core.evolve_leapfrog(s)

            # Copy gauge links and gauge fields from the memory of the GPU
            if t%DTS == 0:
                ui = s.u1.copy()
                ax_ah[t//DTS] = np.apply_along_axis(ln_ah, 1, ui[:, 0, :])
                ax_taylor[t//DTS] = np.apply_along_axis(ln_taylor, 1, ui[:, 0, :])
                        
            pbar.update(1)

    if use_cuda:
        cuda.current_context().deallocations.clear()

    return ax_ah, ax_taylor

#### Run the simulation 

In [6]:
# Supress various horribly long warnings
import warnings
warnings.filterwarnings('ignore')

In [18]:
ax_ah, ax_taylor = simulate()

100%|██████████| 512/512 [02:52<00:00,  2.97it/s]


In [24]:
ax_taylor_comp = np.apply_along_axis(su.get_algebra_factors_from_group_element_approximate, 0, ax_taylor[:,:,:])
ax_ah_comp = np.apply_along_axis(su.get_algebra_factors_from_group_element_approximate, 0, ax_ah[:,:,:])

In [25]:
ax_taylor_comp[1,1,:] - ax_ah_comp[1,1,:] 

array([ 5.22083382e-16,  9.75446383e-05,  2.15475542e-05, -9.75446383e-05,
        1.81232363e-16,  8.31505661e-05, -2.15475542e-05, -8.31505661e-05,
       -4.02775912e-16])